# The C programming language

## Introduction and Motivation

Modern computational physics often demands both rapid prototyping and extreme computational efficiency.
While Python offers clarity and rapid development, its interpreted nature can introduce performance bottlenecks in computation-heavy tasks.
In this section, we discuss the historical context and practical reasons for incorporating the C programming language into your scientific computing toolbox.

### Why C?

When this note is written, C and C++ are the 4th and 2nd popular programming lagnauges on the [TIOBE Index](https://www.tiobe.com/tiobe-index).
There are many reasons contribute to its significance.

![Ken Thompson (left) with Dennis Ritchie (right)](figures/Thompson+Ritchie.jpg)

* **Historical Relevance and Impact**
  * Birth of Unix and C:
    C was developed in the early 1970s by Dennis Ritchie and played a critical role in the development of Unix.
    Its influence persists in virtually every modern operating system and programming language.
  * Foundational Influence:
    Many languages (like C++, Java, and even Python's CPython interpreter) are built on principles originating in C.
    Learning C gives you insight into the low-level operations that drive higher-level languages.

* **Performance Benefits**
  * Compiled Efficiency:
    C is a compiled language, meaning the code is translated directly into machine code before execution.
    This typically results in significant performance gains over interpreted languages.
  * Memory Management:
    With C, you have explicit control over memory allocation and deallocation.
    This low-level control can be crucial for optimizing large-scale simulations and numerical computations in physics.
  * Deterministic Behavior:
    C's simplicity and control mean that you can predict performance characteristics more accurately.
    For simulations or numerical integrations where timing and resource usage are critical, this predictability is a major advantage.

* **Practical Applications in Computational Physics**
  * High-Performance Libraries:
    Many of the core libraries used in computational physics (e.g., BLAS, LAPACK, FFTW) are written in C or have C interfaces.
    Understanding C can help you better leverage and even extend these libraries.
  * Embedded and Real-Time Systems:
    In areas like signal processing, control systems, and experimental setups, C is often the language of choice due to its speed and direct hardware interfacing.
  * Scalable Simulations:
    When simulations require thousands or millions of iterations (for example, in numerical integration of differential equations), the efficiency of C can be the difference between a simulation that runs in seconds versus one that runs in hours.

### When to Use C vs. Python

* **Complementary Roles in Scientific Computing**
  * Rapid Development vs. Execution Speed:
  * Python excels at quickly writing, testing, and iterating on code.
    Its syntax is clean and it offers a rich ecosystem for data manipulation and visualization.
  * C excels at executing performance-critical tasks.
    Once a prototype is validated in Python, rewriting the heavy computations in C (and call it from python) can drastically reduce runtime.

* **Trade-offs and Synergy**
  * Development Trade-offs:
    While Python minimizes the time to prototype, its interpreted nature means that it can be slower for loop-intensive computations or tasks that require fine-grained memory control.
    C, being low-level, requires more careful management of resources, but it pays off when performance is key.
  * Hybrid Approaches:
    In many scientific applications, a hybrid approach is used:
    * C for "Heavy Lifting":
      Writing the performance-critical portions (such as numerical integration, matrix operations, or simulation kernels) in C.
    * Python for Scripting and Visualization:
      Using Python to manage input/output, orchestrate simulations, and visualize results.
    This division of labor allows you to benefit from the best of both worlds—rapid development and high performance.

* **Example**
  * Many of the popular simulation codes are written in C/C++, e.g.,
    [Gadget](https://wwwmpa.mpa-garching.mpg.de/gadget4/),
    [AthenaK](https://github.com/IAS-Astrophysics/athenak).
  * While data visualization are written in python, e.g., [yt](https://yt-project.org/).

## Quick Example

To start with, let's compare directly a forward euler code written in Python and C:

In [ ]:
# This is the simple forward Euler example we used in ODE I:
# https://ua-2025q1-phys305.github.io/14/notes.html#example

from matplotlib import pyplot as plt

def forwardEuler(f, x, t, dt, n):
    T = [t]
    X = [x]
    for i in range(n):
        t += dt
        x += f(x) * dt
        T.append(t)
        X.append(x)
    return T, X

def f(x):
    return x

Tpy, Xpy = forwardEuler(f, 1, 0, 0.1, 20)

plt.plot(Tpy, Xpy, 'o-')
plt.xlabel('t')
plt.ylabel('x')

To translate it to C, it becomes something like:
```C
/* Place the following code in "example.c" */

#include <stdlib.h>
#include <stdio.h>

struct solution {
	double *T;
	double *X;
};

struct solution
forward_Euler(double (*f)(double), double x, double t, double dt, int n)
{
	double *T = (double *)malloc(sizeof(double) * (n+1));
	double *X = (double *)malloc(sizeof(double) * (n+1));

	T[0] = t;
	X[0] = x;

	for(int i = 1; i <= n; ++i) {
		t += dt;
		x += f(x) * dt;
		T[i] = t;
		X[i] = x;
	}

	struct solution s = {T, X};
	return s;
}

void
free_solution(struct solution s)
{
	free(s.T);
	free(s.X);
}

double f(double x)
{
	return x;
}

int main(int argc, char *argv[])
{
	int n = 20;
	struct solution s = forward_Euler(f, 1, 0, 0.1, n);

	for(int i = 0; i <= n; ++i)
		printf("%f %f\n", s.T[i], s.X[i]);

	free_solution(s);

	return 0;
}
```

This is not very elegant...
But before we dive into the details of the code, let's try to run it.

1. Copy and paste the above code block into a file called "example.c".
2. Run `gcc example.c -o example` in your terminal.
   You should a new file `example`.
3. Run the code by `./example`.
   It does not create any plot!
   Instead, just a list of numbers.
4. We may use python to create the plot.
   But this requires first saving the output to a file.
   Run `./example > data.txt`.
   Here, `>` is the bash redirection operator.
   It redirects the standard output of a program to a file.
6. Once "data.txt" is created, run the following python cell:

In [ ]:
from numpy import genfromtxt

data = genfromtxt('data.txt')
TC = data[:,0]
XC = data[:,1]

plt.plot(Tpy, Xpy, 'o-')
plt.plot(TC,  XC,  '.-')
plt.xlabel('t')
plt.ylabel('x')

The C result agrees with the python result, as expected.

Let's now go through the compilation process and the code with more details.

## The C Programming Language

The best places to learn C are:

* [**The C Programming Language**](https://www.amazon.com/Programming-Language-2nd-Brian-Kernighan/dp/0131103628) by Brian W. Kernighan and Dennis M. Ritchie
* The [C-FAQ](https://c-faq.com)
* And high quanlity open source software including the [GNU Scientific Library](https://savannah.gnu.org/git/?group=gsl), [FFTW](https://github.com/FFTW/fftw3), and [Linux Kernel](https://github.com/torvalds/linux).

Many "modern C" books are full of errors, especially memory management.
Avoid them.

![C Book](figures/C-book.jpg)